In [142]:
reset -fs

In [143]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [144]:
from datetime import date, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os

In [145]:
date.today()

datetime.date(2022, 4, 4)

In [146]:
today = date.today()
yesterday = today + timedelta(days = -1)
yesterday_date = '-'.join([str(yesterday.year), str(yesterday.month), str(yesterday.day)])

In [147]:
base = date(2021, 4, 1)
date_list = [base + timedelta(days = x) for x in range(187)]

In [148]:
date_list

[datetime.date(2021, 4, 1),
 datetime.date(2021, 4, 2),
 datetime.date(2021, 4, 3),
 datetime.date(2021, 4, 4),
 datetime.date(2021, 4, 5),
 datetime.date(2021, 4, 6),
 datetime.date(2021, 4, 7),
 datetime.date(2021, 4, 8),
 datetime.date(2021, 4, 9),
 datetime.date(2021, 4, 10),
 datetime.date(2021, 4, 11),
 datetime.date(2021, 4, 12),
 datetime.date(2021, 4, 13),
 datetime.date(2021, 4, 14),
 datetime.date(2021, 4, 15),
 datetime.date(2021, 4, 16),
 datetime.date(2021, 4, 17),
 datetime.date(2021, 4, 18),
 datetime.date(2021, 4, 19),
 datetime.date(2021, 4, 20),
 datetime.date(2021, 4, 21),
 datetime.date(2021, 4, 22),
 datetime.date(2021, 4, 23),
 datetime.date(2021, 4, 24),
 datetime.date(2021, 4, 25),
 datetime.date(2021, 4, 26),
 datetime.date(2021, 4, 27),
 datetime.date(2021, 4, 28),
 datetime.date(2021, 4, 29),
 datetime.date(2021, 4, 30),
 datetime.date(2021, 5, 1),
 datetime.date(2021, 5, 2),
 datetime.date(2021, 5, 3),
 datetime.date(2021, 5, 4),
 datetime.date(2021, 5, 5),

In [149]:
columns = ['date', 't1', 'pitcher', 't2', 'batter', 'inning', 'result', 'pitch_type', 'mph', 'rpm', 'vbreak', 'up_down', 'hbreak', 'left_right', 'count']

Format of rows:
- NULL
- Pitcher
- NULL
- Batter
- Game Pitch
- Pitch number (from pitcher)
- Plate appearance #
- Inning
- Result
- Pitch type
- MPH
- RPM
- Vbreak
- up/down
- Hbreak
- l/r

In [150]:
import pandas as pd

In [151]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

for game_day in date_list:
    target_date = '-'.join([str(game_day.year), str(game_day.month), str(game_day.day)])
    path = 'https://baseballsavant.mlb.com/gamefeed?date=' + target_date + '&chartType=pitch&legendType=pitchName' + \
            '&playerType=pitcher&inning=&count=&pitchHand=&batSide=&descFilter=&ptFilter=&resultFilter=&hf=pitchVelocity#706856'
    driver.get(path)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    df = pd.DataFrame(columns = columns)
    games = soup.find_all('div', {'class': 'game-container step'})
    teams_left = soup.find_all('div', {'class':'team-left'})
    teams_right = soup.find_all('div', {'class':'team-right'})
    for game, team_left, team_right in zip(games, teams_left, teams_right):
        try:
            pitches = game.find('tbody').find_all('tr')
            
            t1 = team_left.find('div', {'class':'team-name'}).text.replace('\n', '').strip()

            t2 = team_right.find('div', {'class':'team-name'}).text.replace('\n', '').strip()
            
        except:
            print('Break')
            break
        count_b = 0
        count_s = 0
        for pitch in pitches[::-1]:
            try:
                row = pitch.find_all('span')

                pitcher = row[1].text
                batter = row[3].text
                inning = int(row[7].text)
                result = row[8].text
                pitch_type = row[9].text
                mph = float(row[10].text)
                rpm = int(row[11].text)

                vbreak = int(row[12].text)
                up_down = row[13].text
                hbreak = int(row[14].text)
                l_r = row[15].text
                entry = [target_date, t1, pitcher, t2, batter, inning, result, pitch_type, mph, rpm, vbreak, up_down, hbreak, l_r, (count_b, count_s)]
                df.loc[len(df)] = entry
            except:
                pass
            if ('Ball' in result) or ('Pitchout' in result): 
                count_b += 1
            elif ('Strike' in result) or ('Missed Bunt' == result) or ('Foul Bunt' == result):
                count_s += 1
            elif 'Foul' in result:
                if count_s < 2:
                    count_s += 1
            if (count_b == 4) or (count_s == 3) or ('In play' in result) or ('Hit by Pitch' == result):
                count_b = 0
                count_s = 0
    df.to_csv(str(game_day.month) + '/' + target_date + '.csv')
driver.close()

/var/folders/4g/z8wcy4nn4vzgq33ts3mh1lm00000gn/T/ipykernel_3445/1253786773.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break
Break


---

In [121]:
df

,date,t1,pitcher,t2,batter,inning,result,pitch_type,mph,rpm,vbreak,up_down,hbreak,left_right,count
0,2021-9-1,Cardinals,Wade Miley,Reds,Tommy Edman,1,Called Strike,Cutter,83.5,2205,36,↓,4,←,"(0, 0)"
1,2021-9-1,Cardinals,Wade Miley,Reds,Tommy Edman,1,"In play, out(s)",Changeup,82.1,1743,37,↓,12,→,"(0, 1)"
2,2021-9-1,Cardinals,Wade Miley,Reds,Paul Goldschmidt,1,Called Strike,Cutter,83.9,2212,35,↓,6,←,"(0, 0)"
3,2021-9-1,Cardinals,Wade Miley,Reds,Paul Goldschmidt,1,Ball,4-Seam Fastball,89.9,2068,21,↓,4,←,"(0, 1)"
4,2021-9-1,Cardinals,Wade Miley,Reds,Paul Goldschmidt,1,"In play, run(s)",Changeup,82.3,1815,33,↓,12,→,"(1, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588,2021-9-1,Marlins,Anthony Bass,Mets,James McCann,9,Ball In Dirt,Slider,87.4,2254,36,↓,3,→,"(0, 2)"
4589,2021-9-1,Marlins,Anthony Bass,Mets,James McCann,9,Foul,Slider,87.1,2218,36,↓,3,→,"(1, 2)"
4590,2021-9-1,Marlins,Anthony Bass,Mets,James McCann,9,Foul,Slider,86.8,2296,38,↓,4,→,"(1, 2)"
4591,2021-9-1,Marlins,Anthony Bass,Mets,James McCann,9,Foul,Slider,86.9,2276,36,↓,1,→,"(1, 2)"


In [122]:
pitch_type = 'Curveball'

In [123]:
leaderboard = df.loc[(df.pitch_type == pitch_type) & (df.result == 'Swinging Strike')].sort_values(by = 'rpm', ascending = False).head()

In [124]:
leader = leaderboard.iloc[0]
query = ' '.join([leader.pitcher, leader.batter, 'inning', str(leader.inning), str(leader['count'][0]), 'ball', str(leader['count'][1]), 'strike'])

In [140]:
query_url = 'https://www.mlb.com/video/search?q=' + leader.pitcher.replace(' ', '+') + '+' + \
    leader.batter.replace(' ', '+') + '+inning+' + str(leader.inning) + '+' + str(leader['count'][0]) + '+ball+' + \
    str(leader['count'][1]) + '+strike&qt=FREETEXT'

In [141]:
driver.get(query_url)

In [132]:

time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [139]:
url_end = soup.find('div', {'id': 'mlb-video-app'}).find('a',{'type': 'primary', 'data-testid': 'link'})['href']
video_url = 'https://www.mlb.com' + url_end

'/video/bryce-harper-strikes-out-swinging-9p0ks4?q=Paolo%20Espino%20Bryce%20Harper%20inning%201%202%20ball%202%20strike&cp=MIXED&qt=FREETEXT&p=0'

In [ ]:
df.vbreak = df.vbreak * ((df.up_down == '↑') -0.5) * 2
df.hbreak = df.hbreak * ((df.left_right == '←') - 0.5) * 2

In [111]:
#for i in range(1,13):
#    os.mkdir(str(i))

TypeError: expected str, bytes or os.PathLike object, not list